In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import sqlite3
from peewee import SqliteDatabase, Model, CharField, IntegerField
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from matplotlib import pyplot as plt
import seaborn as sns


In [ ]:
sqlite_file = 'country_predict.db'
conn = sqlite3.connect(sqlite_file)
query = 'SELECT * FROM CountryData'
data_from_db = pd.read_sql_query(query, conn)
conn.close()
data_from_db.head()

In [ ]:
data_for_france = data_from_db[data_from_db['country']=='France']
data_for_france.head(15)

In [ ]:
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['year']
  ys = series['total_medaille']

  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = data_for_france.sort_values('year', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Année')
_ = plt.ylabel('total de medaille')

Ici nous avons un graphique représentant l'évolution des médailles que la France a gagné a chaque compétition.

In [ ]:
train_data, test_data, train_target, test_target = train_test_split(
    data_for_france[['year']], data_for_france['total_medaille'], test_size=0.2, random_state=42
)

model = LinearRegression()
model.fit(train_data, train_target)

predictions = model.predict(test_data)

next_year = 2024

prediction_next_JO = model.predict([[next_year]])
print(f'Prédiction pour les JO {next_year}: {prediction_next_JO[0]} médailles')


La prédiction concernant le nombre de médailles que la France devrait gagner lors des prochains JO est donc de 35.

In [ ]:
features = ['year', 'country']
target = 'total_medaille'

train_data, test_data, train_target, test_target = train_test_split(
    data_from_db[features],
    data_from_db[target],
    test_size=0.2,
    random_state=42
)

model = RandomForestRegressor()
model.fit(train_data, train_target)

predictions = model.predict(test_data)


future_data = pd.DataFrame({'year': [2024], 'country_noc': ['FRA']})
future_predictions = model.predict(future_data)

predicted_countries = data_from_db[data_from_db['year'] == 2024]
predicted_countries['predicted_medals'] = model.predict(predicted_countries[features])
countries_with_at_least_2_medals = predicted_countries[predicted_countries['predicted_medals'] >= 2]

print('Pays avec au moins 2 médailles prédites pour les prochains JO :')
print(countries_with_at_least_2_medals[['country_noc', 'predicted_medals']])

J'ai eu quelques soucis avec le code ci dessous. Je pense que cela est dues au fait que les données soient des chaînes de caractères et non des nombres, mais je n'ai pas réussi à le corriger en essayant différentes choses. J'ai donc réussi à avoir la prédictions du nombre de médailles pour la France, mais je n'ai pas réussi les 2 autres prédictions. Je laisse le code qui ne fonctionne pas afin de montrer quand même ce que j'avais fait.
Le dataset séléctionné au départ n'était pas bon, et j'ai eu quelques erreurs à cause de ca. J'avais du en changer et refaire mes prévisions.

6 - D'près l'article proposé, on peut voir qu'il est très compliqué de pouvoir prédire avec exactitude des résultats sur le théme des Jo, en particulier à cause d'événements extérieur qui peuvent interférer avec les résultats rééls. Par exemple le COVID dans ce cas précis. Les 20 premières nations ont eu des résultats innatendu par rapport à ce qui était prédit. Cela pourrait être également le cas pour les prochains JO avec par exemple la guerre en Ukraine ou la Russie et l'Ukraine (et probablement d'autres pays) seront impactés par la guerre, et n'auront pas les résultats qui auraient pu être prédit par rapport aux données des derniers JO Olympiques.

7 - La conclusion de cette étude, et que l'on peut avoir une certaines tendances qui peut ressortir, mais il est assez compliqué d'avoir des résultats vraiment fiable. Beaucoup de facteur extérieur sont à prendre en compte, et certains ne peuvent être intégré à notre modéle de predictions.

8 - Pour ce projet, le Big Data n'est pas vraiment utile. Les quantités de données à traiter ne sont pas assez importantes pour avoir besoin d'outils supplémentaires. Le Big Data devient utile lorsque l'on a des quantités de données bien plus conséquentes à traiter que celles ci.